In [15]:
%tensorflow_version 1.x
import tensorflow as tf
print(tf.__version__)

TensorFlow 1.x selected.
1.15.2


In [3]:
# 首先克隆一下官方库
!git clone https://github.com/tensorflow/nmt/

Cloning into 'nmt'...
remote: Enumerating objects: 1283, done.
remote: Total 1283 (delta 0), reused 0 (delta 0), pack-reused 1283
Receiving objects: 100% (1283/1283), 1.24 MiB | 19.84 MiB/s, done.
Resolving deltas: 100% (918/918), done.


In [4]:
# 查看一下数据加载脚本
!cat /content/nmt/nmt/scripts/download_iwslt15.sh

#!/bin/sh
# Download small-scale IWSLT15 Vietnames to English translation data for NMT
# model training.
#
# Usage:
#   ./download_iwslt15.sh path-to-output-dir
#
# If output directory is not specified, "./iwslt15" will be used as the default
# output directory.
OUT_DIR="${1:-iwslt15}"
SITE_PREFIX="https://nlp.stanford.edu/projects/nmt/data"

mkdir -v -p $OUT_DIR

# Download iwslt15 small dataset from standford website.
echo "Download training dataset train.en and train.vi."
curl -o "$OUT_DIR/train.en" "$SITE_PREFIX/iwslt15.en-vi/train.en"
curl -o "$OUT_DIR/train.vi" "$SITE_PREFIX/iwslt15.en-vi/train.vi"

echo "Download dev dataset tst2012.en and tst2012.vi."
curl -o "$OUT_DIR/tst2012.en" "$SITE_PREFIX/iwslt15.en-vi/tst2012.en"
curl -o "$OUT_DIR/tst2012.vi" "$SITE_PREFIX/iwslt15.en-vi/tst2012.vi"

echo "Download test dataset tst2013.en and tst2013.vi."
curl -o "$OUT_DIR/tst2013.en" "$SITE_PREFIX/iwslt15.en-vi/tst2013.en"
curl -o "$OUT_DIR/tst2013.vi" "$SITE_PREFIX/iwslt15.en-vi/tst2013

In [5]:
# 运行脚本，下载数据
!/content/nmt/nmt/scripts/download_iwslt15.sh ./data

mkdir: created directory './data'
Download training dataset train.en and train.vi.
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 12.9M  100 12.9M    0     0  15.5M      0 --:--:-- --:--:-- --:--:-- 15.5M
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 17.2M  100 17.2M    0     0  18.8M      0 --:--:-- --:--:-- --:--:-- 18.8M
Download dev dataset tst2012.en and tst2012.vi.
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  136k  100  136k    0     0   511k      0 --:--:-- --:--:-- --:--:--  511k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed


In [6]:
# 看一下英文训练集的前10行
!head -10 ./data/train.en

Rachel Pike : The science behind a climate headline
In 4 minutes , atmospheric chemist Rachel Pike provides a glimpse of the massive scientific effort behind the bold headlines on climate change , with her team -- one of thousands who contributed -- taking a risky flight over the rainforest in pursuit of data on a key molecule .
I &apos;d like to talk to you today about the scale of the scientific effort that goes into making the headlines you see in the paper .
Headlines that look like this when they have to do with climate change , and headlines that look like this when they have to do with air quality or smog .
They are both two branches of the same field of atmospheric science .
Recently the headlines looked like this when the Intergovernmental Panel on Climate Change , or IPCC , put out their report on the state of understanding of the atmospheric system .
That report was written by 620 scientists from 40 countries .
They wrote almost a thousand pages on the topic .
And all of tho

In [7]:
# 看一下越南语训练集的前10行
!head -10 ./data/train.vi

Khoa học đằng sau một tiêu đề về khí hậu
Trong 4 phút , chuyên gia hoá học khí quyển Rachel Pike giới thiệu sơ lược về những nỗ lực khoa học miệt mài đằng sau những tiêu đề táo bạo về biến đổi khí hậu , cùng với đoàn nghiên cứu của mình -- hàng ngàn người đã cống hiến cho dự án này -- một chuyến bay mạo hiểm qua rừng già để tìm kiếm thông tin về một phân tử then chốt .
Tôi muốn cho các bạn biết về sự to lớn của những nỗ lực khoa học đã góp phần làm nên các dòng tít bạn thường thấy trên báo .
Có những dòng trông như thế này khi bàn về biến đổi khí hậu , và như thế này khi nói về chất lượng không khí hay khói bụi .
Cả hai đều là một nhánh của cùng một lĩnh vực trong ngành khoa học khí quyển .
Các tiêu đề gần đây trông như thế này khi Ban Điều hành Biến đổi khí hậu Liên chính phủ , gọi tắt là IPCC đưa ra bài nghiên cứu của họ về hệ thống khí quyển .
Nghiên cứu được viết bởi 620 nhà khoa học từ 40 quốc gia khác nhau .
Họ viết gần 1000 trang về chủ đề này .
Và tất cả các trang đều được xem 

In [8]:
# 看一下英文词汇的前10行
!head -10 ./data/vocab.en

<unk>
<s>
</s>
Rachel
:
The
science
behind
a
climate


In [16]:
# 进行训练
# 不同的attention的方式
# 源语言是越南语，目标语言是英语，这是后缀
# 词表
# 训练集
# 验证集
# 测试集
# 模型输出位置 
# 训练步数
!python3 -m nmt.nmt.nmt \
    --attention=scaled_luong \
    --src=vi --tgt=en \
    --vocab_prefix=./data/vocab  \
    --train_prefix=./data/train \
    --dev_prefix=./data/tst2012  \
    --test_prefix=./data/tst2013 \
    --out_dir=/tmp/nmt_attention_model \
    --num_train_steps=12000 \
    --steps_per_stats=100 \
    --num_layers=2 \
    --num_units=128 \
    --dropout=0.2 \
    --metrics=bleu


The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

W0129 06:26:09.327220 139824990775168 lazy_loader.py:50] 
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

I0129 06:26:10.478821 139824990775168 utils.py:157] NumExpr defaulting to 2 threads.
# Job id 0

W0129 06:26:11.447041 139824990775168 module_wrapper.py:139] From /content/nmt/nmt/nmt.py:629: The name tf.Session i

In [17]:
# 查看一下可以调整的参数
!python3 -m nmt.nmt.nmt --help

usage: nmt.py [-h] [--num_units NUM_UNITS] [--num_layers NUM_LAYERS]
              [--num_encoder_layers NUM_ENCODER_LAYERS]
              [--num_decoder_layers NUM_DECODER_LAYERS]
              [--encoder_type ENCODER_TYPE] [--residual [RESIDUAL]]
              [--time_major [TIME_MAJOR]]
              [--num_embeddings_partitions NUM_EMBEDDINGS_PARTITIONS]
              [--attention ATTENTION]
              [--attention_architecture ATTENTION_ARCHITECTURE]
              [--output_attention [OUTPUT_ATTENTION]]
              [--pass_hidden_state [PASS_HIDDEN_STATE]]
              [--optimizer OPTIMIZER] [--learning_rate LEARNING_RATE]
              [--warmup_steps WARMUP_STEPS] [--warmup_scheme WARMUP_SCHEME]
              [--decay_scheme DECAY_SCHEME]
              [--num_train_steps NUM_TRAIN_STEPS]
              [--colocate_gradients_with_ops [COLOCATE_GRADIENTS_WITH_OPS]]
              [--init_op INIT_OP] [--init_weight INIT_WEIGHT] [--src SRC]
              [--tgt TGT] [--train_pr

In [32]:
# 训练好的模型进行测试，可以这样做
# 首先在data下新建一个文件，比如my_infer_file.vi
# 然后从tst2013.vi复制一些句子到其中，用作要被翻译的句子。

In [30]:
!python3 -m nmt.nmt.nmt \
    --out_dir=/tmp/nmt_attention_model \
    --vocab_prefix=./data/vocab  \
    --inference_input_file=./data/my_infer_file.vi \
    --inference_output_file=./output_infer


The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

W0129 07:41:36.001784 139642247858048 lazy_loader.py:50] 
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

I0129 07:41:36.438718 139642247858048 utils.py:157] NumExpr defaulting to 2 threads.
# Job id 0

W0129 07:41:36.705005 139642247858048 module_wrapper.py:139] From /content/nmt/nmt/nmt.py:629: The name tf.Session i

In [31]:
# 查看一下生成的文件
!cat ./output_infer # To view the inference as output

When I was a little kid , I think the <unk> of the world is the best of the world and I &apos;m going to sing the &quot; We have no <unk> . &quot;
I was so proud of the country .
In the school , we have a lot of time to learn the <unk> of the <unk> of the <unk> <unk> , but the <unk> of the world , the <unk> of the world , the <unk> of the <unk> , the <unk> of the <unk> , the <unk> of the <unk> , the <unk> of the <unk> of the <unk> of the <unk> of the <unk> of the <unk> of the <unk> of the <unk> of the <unk> of the <unk> of the <unk> of the <unk> of the <unk> of the <unk> of the <unk> of the <unk> of the
Even though I &apos;ve been wondering that the world is not going to know how to the outside of the world , but I think I &apos;d be able to live the whole life of the <unk> , until the <unk> of the <unk> .
When I was seven , I saw the <unk> of the people who <unk> the first time , but I think the life of my life is completely <unk> .
My family is not poor , and I have not been a lot of